In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# Tu si treba vybrat jedno GPU (cize bud 0 alebo 1)
# Je to cislovane naopak ako v nvidia-smi, because ..., that's why
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
import multiprocessing
os.environ["OMP_NUM_THREADS"] = str(multiprocessing.cpu_count() - 1)
print(str(multiprocessing.cpu_count() - 1))
import torch
torch.cuda.empty_cache()

7


In [3]:
import torch
import torch.nn as nn
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import copy
import sys

import random
import numpy as np
import torch
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable

import sys
import numpy as np
import timm
from functools import partial
from tqdm.notebook import tqdm

In [4]:
!pip install --upgrade jupyter ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter nbextension install --py widgetsnbextension
!jupyter lab --version
!jupyter --version
!pip install scikit-learn

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/jlichmanova/master-thesis-pruning-dnn/mip-env/lib/python3.10/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/jlichmanova/master-thesis-pruning-dnn/mip-env/lib/python3.10/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/jlichmanova/master-thesis-pruning-dnn/mip-env/lib/python3.10/site-packages/jupyterlab/debuglog.py:56: UserWarning: See th

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


4.2.6


Selected Jupyter core packages...


IPython          : 8.30.0
ipykernel        : 6.29.5


ipywidgets       : 8.1.7
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : 2.14.2


jupyterlab       : 4.2.6
nbclient         : 0.10.1


nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : 7.2.2
qtconsole        : not installed
traitlets        : 5.14.3


In [5]:
IMAGENET_PATH = "/data/imagenet/imagenet/"

sys.path.append(os.path.abspath('../clustering_algorithms/correlation_algorithms/whole_correlation'))
print(os.listdir('/home/jlichmanova/master-thesis-pruning-dnn/clustering_algorithms/correlation_algorithms/whole_correlation'))

from cosine_dist_mip import compute_clustering_permutations

['runs', 'output', 'cosine_dist_mip.py', 'abs_output', '__pycache__']


In [6]:
sys.path.append(os.path.abspath('../clustering_algorithms/spectral_clustering'))
from spectral_KNN_balanced import compute_clustering_permutations_SPECTRAL_KNN_balanced

In [11]:
def random_seed(seed=42, rank=0):
    torch.manual_seed(seed + rank)
    np.random.seed(seed + rank)
    random.seed(seed + rank)

random_seed(47)

In [12]:
device = torch.device("cuda")
amp_autocast = partial(torch.autocast, device_type=device.type, dtype=torch.float16)

In [13]:
# Very small models
#model_name = "test_vit3.r160_in1k" # accuracy 0.56
model_name = "test_vit2.r160_in1k" # accuracy 0.42
#model_name = "test_vit.r160_in1k" 

# Bigger models
#model_name = "vit_wee_patch16_reg1_gap_256.sbb_in1k" # val acc 0.80272
#model_name = "vit_medium_patch16_reg4_gap_256.sbb_in1k" #val acc 0.83452

# Even bigger
# See bigger than tiny from: https://huggingface.co/timm/convnext_tiny.fb_in1k
# Or others at https://huggingface.co/timm/vit_wee_patch16_reg1_gap_256.sbb_in1k

model = timm.create_model(model_name, pretrained=True)
model.cuda() 
print(model)

sum(p.numel() for p in model.parameters())

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 64, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=64, out_features=192, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=64, out_features=192, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
     

455720

In [14]:
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input, target in tqdm(val_loader):
            with amp_autocast():
                output = model(input)
                correct += (target == output.argmax(dim=1)).sum().item()
                total += target.numel()
    print("val acc", correct / total)

In [15]:
data_config = timm.data.resolve_data_config(model=model)

print(data_config)

val_dataset = timm.data.create_dataset(
    name="imagenet",
    split="validation",
    root=IMAGENET_PATH
)

val_loader = timm.data.create_loader(
    val_dataset,
    input_size=data_config['input_size'],                                                 
    batch_size=64,
    use_prefetcher=True,                                                       
    interpolation=data_config['interpolation'],                                           
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=5,                                                             
    crop_pct=data_config["crop_pct"],
    crop_mode=data_config['crop_mode'],                                                   
    crop_border_pixels=False,  
    pin_memory=True,
    device=device,
)

for bx, by in val_loader:
    print(bx.shape, by.shape)
    print(by)
    break

{'input_size': (3, 160, 160), 'interpolation': 'bicubic', 'mean': (0.5, 0.5, 0.5), 'std': (0.5, 0.5, 0.5), 'crop_pct': 0.95, 'crop_mode': 'center'}


torch.Size([64, 3, 160, 160]) torch.Size([64])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')


In [16]:
train_dataset = timm.data.create_dataset(
    name="imagenet",
    split="train",
    root=IMAGENET_PATH
)

# TODO: augmentations
train_loader = timm.data.create_loader(
    train_dataset,
    input_size=data_config['input_size'],                                                 
    batch_size=64,
    use_prefetcher=True,                                                       
    interpolation="random",  
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=5,                                                             
    crop_pct=data_config["crop_pct"],
    crop_mode=data_config['crop_mode'],                                                   
    crop_border_pixels=False,  
    pin_memory=True,
    device=device,
    is_training=True,
)

In [17]:
# Run validation on full model (reports number in range 0-1)

#validate(model, val_loader)

In [18]:
# Function for gathering input statistics for a layer
def update_cov(m, i, o):
    x = i[0].detach().flatten(0, -2).float()
    with torch.autocast(device_type="cuda", enabled=False):
        m.XX.data += x.square().sum(dim=0)
        
    x_out = o.detach().view(-1, o.shape[-1])
    with torch.autocast(device_type="cuda", enabled=False):
        m.output_stats.data += x_out.square().sum(dim=0)
        
# Selection functions for picking which layers to compress
# In general, we do not compress first and last layer
# Name of the layer is n.n2 (where there are no dots in n2)
# Actual layer is m
select_all_linear = lambda n, n2, m: type(m) == nn.Linear and "head" not in n and "head" not in n2
select_only_attn_proj = lambda n, n2, m: type(m) == nn.Linear and "head" not in n and "head" not in n2 and "proj" in n2
select = lambda n, n2, m: type(m) == nn.Linear and "blocks.1" in n and "proj" in n2

def process_model(model_name=model_name, replace_filter=select_all_linear, 
                  replace_fn=lambda x: x, finetune_epochs=2, finetune_maxlr=1e-5,
                  wanda=False):
    """
    :model_name: Name of model
    :replace_filter: Which layers to replace
    :replace_fn: Takes a layer, outputs new layer (or sequence of layers)
    :finetune_epochs: How many epochs to use for finetuning
    :finetune_maxlr: Maximal learning rate for finetuning
    :wanda: Whether to collect wanda statistics (if False, norms of inputs will be one)
    """
    model = timm.create_model(model_name, pretrained=True)
    model.cuda()
    
    #print("pars before", sum(p.numel() for p in model.parameters()))
    
    # need to dump, because changing during iteration would make a mess
    to_change = []
    
    for n, m in model.named_modules():
        for n2, m2 in m.named_modules():
            if "." not in n2 and replace_filter(n, n2, m2) and len(n2) > 0:
                m2.XX = torch.zeros((m2.weight.shape[1]), device=m2.weight.device)
                m2.output_stats = torch.zeros((m2.weight.shape[0]), device=m2.weight.device)
                m2.register_forward_hook(update_cov)
                to_change.append((m, n2, m2))      
    
    print("activations...")
    step = 0
    model.eval()
    with torch.no_grad():
        for input, _ in train_loader:
            with amp_autocast():
                model(input)
            step += 1
            if step >= 100:
                break
    
    for m, n2, m2 in to_change:
        setattr(m, n2, replace_fn(m2))
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=finetune_maxlr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=len(train_loader)*finetune_epochs)
    scaler = torch.GradScaler("cuda")

    torch.save(model, "test_vit2.r160_in1k_only_monarch_activations_model.pth")
    log_file = model_name + "_only_monarch_activations_12epoch_training_log.txt"

    with open(log_file, "w") as f:
        for ep in range(finetune_epochs):
            start_time = time.time()
            
            model.train()
            loss_sum = 0
            loss_cc = 0
            correct = 0
            total = 0
    
            for input, target in (pbar := tqdm(train_loader)):
                with amp_autocast():
                    output = model(input)
                    loss = F.cross_entropy(output, target)
    
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()
    
                loss_sum += loss.item()
                loss_cc += 1
            
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for input, target in tqdm(val_loader):
                    with amp_autocast():
                        output = model(input)
                        correct += (target == output.argmax(dim=1)).sum().item()
                        total += target.numel()

            epoch_time = time.time() - start_time
            val_acc = correct / total
            
            print("val acc", val_acc)
            print(f"Epoch {ep+1}/{finetune_epochs}, Loss: {loss_sum/loss_cc:.4f}, Accuracy: {val_acc:.4f}%, Time: {epoch_time:.2f} sec\n")
            f.write(f"Epoch {ep+1}/{finetune_epochs}, Loss: {loss_sum/loss_cc:.4f}, Accuracy: {val_acc:.4f}%, Time: {epoch_time:.2f} sec\n")
            f.flush()
    #torch.save(model, model_name + "SPEC_KNN2_balanced1_activations_12Epoch_model.pth")

In [19]:
class PermutationLayer(nn.Module):
    # permutation 1D tensor 
    # dim -2 rows
    # dim -1 columns
    def __init__(self, permutation, dim=-1):
        super().__init__()
        self.register_buffer("permutation", permutation)
        self.dim = dim  #along second-to-last dimension

    def forward(self, x):
        #print(f"Permuting along dimension {self.dim}, input shape: {x.shape}")
        #print("Before x", x)
        x = x.index_select(self.dim, self.permutation)
        #print("Permutated x", x)
        #print(f"Output shape: {x.shape}")
        return x

class BlockLinear2(nn.Module):
    def __init__(self, nb=4, bs=(128,512), bias=False, dbg=False):
        super().__init__()
        
        self.weight = nn.Parameter(torch.zeros(nb, bs[0], bs[1]))
        self.bias = None
        if bias:
            self.bias = nn.Parameter(torch.zeros(bs[1]))
    
    def extra_repr(self):
        return "w_shape %s" % str(self.weight.shape)
    
    def forward(self, x):
        orig_shape = x.shape
        x = x.flatten(0, -2)
        x = x.reshape(x.shape[0], self.weight.shape[0], -1).permute(1, 0, 2)
        x = x.matmul(self.weight)
        x = x.permute(1, 0, 2)
        x = x.reshape(tuple(list(orig_shape[0:-1]) + [-1]))
        if self.bias is not None:
            x = x + self.bias
        return x

class MonarchPerm2(nn.Module):
    def __init__(self, nb=4, size=128):
        super().__init__()
        out = []
        perm_base = torch.arange(size)
        parts = perm_base.chunk(nb*nb)
        for i in range(nb):
            out += parts[i::nb]
        self.perm = torch.cat(out).cuda()
        
    def forward(self, x):
        orig_shape = x.shape
        x = x.flatten(0, -2)
        out = x.T[self.perm].T
        return out.reshape(orig_shape)
    
class LambdaLayer(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        
    def forward(self, x):
        return self.f(x)

def replace_with_monarch(m, n_blocks=4, rank=None):
    assert m.weight.shape[0] % n_blocks == 0
    assert m.weight.shape[1] % n_blocks == 0
    if rank == None:
        # ~50% of original weights
        bs = m.weight.shape[0] // n_blocks, m.weight.shape[1] // n_blocks
        rank = int((bs[0] * bs[1]) / (bs[0] + bs[1]) / 2)
        print("auto rank", rank)
    
    in_norm = m.XX.sqrt() + 1e-8
    in_norm = in_norm / in_norm.mean()
    print(m, in_norm.amin(), in_norm.amax())

    out_norm = m.output_stats.sqrt() + 1e-8
    out_norm = out_norm / out_norm.mean()
    print(m, out_norm.amin(), out_norm.amax())
    
    W = m.weight.detach()
    W = W * in_norm  
    
    mid = rank * n_blocks

    s0 = W.shape[0] // n_blocks
    s1 = W.shape[1] // n_blocks
    svd_err = 0
    
    weights_cpu = m.weight.detach().cpu()
    
    layer2 = nn.Sequential(
        BlockLinear2(n_blocks, (W.shape[1]//n_blocks, mid)), 
        MonarchPerm2(n_blocks, mid*n_blocks),
        BlockLinear2(n_blocks, (mid, W.shape[0]//n_blocks), bias=(m.bias is not None)),
    ).cuda()
    
    print("density", sum(p.numel() for p in layer2.parameters()) / W.numel())
    
    triv_err = 0
    for i in range(n_blocks):
        for j in range(n_blocks):
            part = W[i*s0:i*s0+s0, j*s1:j*s1+s1]

            U, s, Vh = torch.linalg.svd(part, full_matrices=False)
            s = s[:rank]
            U = U[:,:rank] * s.sqrt()
            Vh = Vh[:rank] * s.sqrt().unsqueeze(1)
            svd_err += (part - (U @ Vh)).square().sum().item()
            triv_err += part.square().sum().item()
            assert layer2[0].weight.data[j, :,rank*i:rank*i+rank].numel() > 0
            assert layer2[2].weight.data[i, rank*j:rank*j+rank,:].numel() > 0
            layer2[0].weight.data[j, :, rank*i:rank*i+rank] = (Vh / in_norm[j*s1:j*s1+s1]).T
            layer2[2].weight.data[i, rank*j:rank*j+rank] = U.T

    #print(P_rows.shape, P_columns.shape)
    print(s0, s1)
    print("svd", svd_err, "triv", triv_err)
    
    if m.bias is not None:
        layer2[2].bias.data = m.bias
    
    test = torch.eye(m.weight.shape[1], device="cuda") 
    if m.bias is not None:
        print("err", (layer2(test) - m(test)).square().mean().item(), "triv err", (m(test) - m.bias.detach()).square().mean().item())
    else:
        print("err", (layer2(test) - m(test)).square().mean().item(), "triv err", (m(test)).square().mean().item())
        
    test = torch.eye(m.weight.shape[1], device="cuda")
    if m.bias is not None:
        print("err norm", (layer2(test) - m(test)).square().mean().item(), "triv err", (m(test) - m.bias.detach()).square().mean().item())
    else:
        print("err norm", (layer2(test) - m(test)).square().mean().item(), "triv err", (m(test)).square().mean().item())
    
    return layer2

process_model(replace_fn=replace_with_monarch, finetune_epochs=12, finetune_maxlr=1e-4)

activations...


auto rank 6
Linear(in_features=64, out_features=192, bias=True) tensor(0.5329, device='cuda:0') tensor(2.2839, device='cuda:0')
Linear(in_features=64, out_features=192, bias=True) tensor(nan, device='cuda:0') tensor(nan, device='cuda:0')
center [[ 1.07890591e-02  6.87336596e-03  1.34044103e-02  1.88125181e-03
  -3.33762020e-02  7.38448370e-03 -7.02556362e-03 -1.33237615e-02
  -2.62128264e-02 -2.83371396e-02  2.38152524e-03  6.57103816e-03
  -4.47729081e-02 -7.72414869e-03  1.26086911e-02  1.21905813e-02
   4.60607186e-03  1.24196121e-02  1.81530733e-02  1.55083360e-02
   3.05198506e-03  1.24508794e-03  4.18261290e-02  1.39019091e-03
  -6.89732563e-03  4.30558482e-03 -4.30705579e-04  7.48396479e-03
  -3.24457721e-03  3.99472890e-03 -1.34533504e-02  1.74301621e-02
   6.51055202e-03  2.93629859e-02 -2.85392720e-02  7.39681162e-03
  -1.81634575e-02 -4.36991919e-03 -6.93512103e-03 -9.77081014e-04
   7.86657538e-03 -8.80223815e-05 -3.66918673e-03 -2.31659953e-02
   7.38520594e-03  2.28334405

/tmp/ipykernel_1246525/1660914226.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  P_rows = torch.tensor(P_rows, device="cuda").clone().detach()
/tmp/ipykernel_1246525/1660914226.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  P_columns = torch.tensor(P_columns, device="cuda").clone().detach()


density 0.50390625
48 16
svd 16.755199640989304 triv 90.34008133411407


err 0.0017430027946829796 triv err 0.006288039498031139
err norm 0.0017430027946829796 triv err 0.006288039498031139
auto rank 4
Linear(in_features=64, out_features=64, bias=True) tensor(0.4321, device='cuda:0') tensor(3.5739, device='cuda:0')
Linear(in_features=64, out_features=64, bias=True) tensor(0.2031, device='cuda:0') tensor(4.1403, device='cuda:0')
center [[-9.6359877e-03  2.1787873e-03  2.3963165e-03  8.4807789e-03
  -6.9155484e-03 -2.4420887e-02 -6.1471076e-03  1.1713477e-02
  -9.1292886e-03 -7.5404714e-03 -5.5064005e-03 -1.7595381e-03
   2.9386734e-03 -9.3359528e-03 -2.0425260e-02  1.1262695e-02
   7.7356648e-04  7.4292667e-04 -6.5645989e-04  1.6647430e-02
   7.5081666e-03 -6.6647539e-03 -2.7309250e-02  3.9814087e-03
  -3.3768515e-06  2.6811822e-03 -2.7085575e-03 -9.9901306e-03
  -7.4947379e-03 -9.3335941e-05  3.9745523e-03  1.2983958e-02
   2.7824179e-03 -7.2099362e-03 -1.3430381e-02 -1.4132818e-02
  -9.4829313e-03 -7.2787371e-03 -1.4829115e-02  1.8279591e-03
  -1.0110389e-

...

  0%|          | 0/782 [00:00<?, ?it/s]

val acc 0.34884
Epoch 11/12, Loss: 3.3632, Accuracy: 0.3488%, Time: 1432.29 sec



  0%|          | 0/20018 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

val acc 0.3493
Epoch 12/12, Loss: 3.3444, Accuracy: 0.3493%, Time: 1422.67 sec

